<a href="https://colab.research.google.com/github/shaniklein/Machine-learning-for-Communication-Networks-and-Systems/blob/main/assignment1_ML_comm_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 1 - Machine learning for Communication Networks and Systems
###By Shani Klein 205781909
This is my code for Exercise 1 in the course Machine learning for Communication Networks and Systems in BGU university

We were given a set of rules in a file. Each row describe a different rule. The structure of the a rule is as follows (the @ sign is the first character of each rule): @SourceIP/sourceMask DestinationIP/destinationMask sourcePort destinationPort protocol The file is given in a TSV format .




# Question number 1

# Getting The Data: 
In this section I upload the file "rule01.tsv" which contains set of rules for 
the assignment. 
if You choose to run this code on your computer:
*   Before you run the code below, ensure you have downloaded the file to your computer.
*   When prompted, browse to the file's location on your computer and upload the file.

Warning: The file upload function will only show the file selection dialog for 30 seconds. After that time it will close and cause an error because a file was not specified. This timeout prevents the file dialog from blocking events in Colab. If you experience this timeout select the dataset file sooner. It may help to move the dataset file to an easier to specify location such as your desktop.



In [ ]:
from google.colab import files
uploaded = files.upload()
rules = next(iter(uploaded.keys()))
print('User uploaded file "{name}" with length {length} bytes'.format(
      name=rules, length=len(uploaded[rules])))


Saving rule01.tsv to rule01.tsv
User uploaded file "rule01.tsv" with length 47001 bytes


## Load the data into a pandas dataframe
I chose to work with pandas dataframe , because it let's us easily review and manipulate the data. 
In addition , we've been told in class that we are only interested in the Sorcse IP , Destenation IP and the corresponding number of wildcards so this is the data I collected from the file of rules

In [ ]:
import pandas as pd
import math 
import numpy as np
# read the tsv data
# data = pd.read_csv("rule01.tsv", header=None)
data = pd.read_csv(rules, header=None) 
# data manipulation to remain only with the columns we want which are  SrcIP, DstIP and number of important bits
data=data[0].str.split("/", n = 2, expand = True)
bit=data[2].str.split(" ", n = 2, expand = True)
bit=bit[0]
newdata=data[1].str.split(" ", n = 2, expand = True)
data[1]=newdata[0]
data[2]=newdata[1]
data[3]=bit
newdata=data[0].str.split("@", n = 2, expand = True)
data[0]=newdata[1]

## Visualize the rules data
This will give us an understanding of the data we are working with.

In [ ]:
# Visualize the dataset
# turn on option to display all columns, otherwise some columns may be hidden
pd.set_option('display.max_columns', None)  
# print the column names and first 200 rows of data so we can see the important bits is not always 32 bits
data.head(210)

,0,1,2,3
0,180.171.147.187,32,83.167.110.154,32
1,180.171.147.161,32,86.109.168.32,32
2,180.171.147.254,32,85.199.42.4,32
3,180.171.147.63,32,25.178.135.86,32
4,180.171.147.89,32,25.184.48.239,32
...,...,...,...,...
205,85.52.83.70,31,199.45.140.2,32
206,85.52.83.112,31,199.45.140.2,32
207,85.52.83.98,31,25.184.48.129,32
208,85.52.83.98,31,199.39.63.180,32


## Convert SrcIP and DstIP to be represented in Bits
As we can see in the Visualization of the data, the IPs are reprsented in the form we know, but we want to convert the IPs to be represented as bits to build our tree, in addition we would like to would like to work with an array of 64 bits for each rule (which contains both source and destination IP.

In [ ]:
#define an helper method to convert each IP adress to be representes in bits
def ip2bin(ip):
  # first- get rid of the dots in the ip address
    octets = map(int, ip.split('/')[0].split('.')) # '1.2.3.4'=>[1, 2, 3, 4]
    binary = '{0:08b}{1:08b}{2:08b}{3:08b}'.format(*octets)
    range = int(ip.split('/')[1]) if '/' in ip else None
    return binary[:range] if range else binary

In [ ]:
def find_wild_card(ip_in_bits,mask):
  for i in range(32):
    if i>=int(mask):
      ip_in_bits=list(ip_in_bits)
      ip_in_bits[i]='*'
      result=str()
      for bit in ip_in_bits:
        result+=bit
      ip_in_bits=result
  return ip_in_bits

In [ ]:
# convert source IP to be represented as 32 bits
data[0] = data[0].apply(lambda x:ip2bin(x))
# put '*' on wild cards
for i in range(len(data[0])):
  data[0][i]=find_wild_card(data[0][i],data[1][i])

# same procedure for destip
data[2]= data[2].apply(lambda x:ip2bin(x))
for i in range(len(data[2])):
  data[2][i]=find_wild_card(data[2][i],data[2][i])

# concate source IP and dest IP to be one array of 64 bits
data[0]=data[0]+data[2]
# we have reduntent column so we drop it from the data
data=data.drop(data.columns[2], axis=1)
data[4] = np.arange(len(data))
# # we add relevent headers to data so it will be easier to access each column
data.columns= ["IP", "s_wildCard", "d_wildCard","rule_number"]


## Visualize the rules data in Binary representaion
let's look how our data is looks right now.

In [ ]:
# Visualize the dataset
# turn on option to display all columns, otherwise some columns may be hidden
pd.set_option('display.max_columns', None)  
# print the column names and first 210 rows of data
data.head(720)

,IP,s_wildCard,d_wildCard,rule_number
0,1011010010101011100100111011101101010011101001...,32,32,0
1,1011010010101011100100111010000101010110011011...,32,32,1
2,1011010010101011100100111111111001010101110001...,32,32,2
3,1011010010101011100100110011111100011001101100...,32,32,3
4,1011010010101011100100110101100100011001101110...,32,32,4
...,...,...,...,...
715,01010101001101000101001*********10011011001101...,23,16,715
716,01010101001101000101001*********01001001000111...,23,16,716
717,01010101001101000101001*********10110111001010...,23,16,717
718,01010101001101000101001*********10110110101111...,23,16,718


##Helper Method to get all rows of the data

In [ ]:
# helper method to get all rows in the data as a dictionary with the column names as keys
def get_data_rows(my_data): 
  rows =[] 
  # Iterate over each row 
  for index, row in my_data.iterrows(): 
      # Create list for the current row 
      my_list ={"IP":row["IP"],"s_wildCard": row["s_wildCard"],"d_wildCard":row["d_wildCard"],"rule_number":row["rule_number"]}
      # append the list to the final list 
      rows.append(my_list) 
  return rows


# Getting Started with Building our Decision Tree
Now that the data is in right form we can use the data and start building out decision tree

##Criteria of maximum information gain
First we will we will find the analytical expression of the information gain (IG) for each rule Ri given the bit bj . so we will have the maximum information gain as a cretiria to splitting the data in the most efficient way.

### Update number of occurrences for each rule depending on the don't cares 
We want to update the number of occurrences for each rule depending on the don't cares meaning,if for example, rule x has 2 don't cares we would refer the rule like the data has 2 in the power of the don't cares occurrences of x rule

In [ ]:
def update_counts(rows,bit):
  """Counts the number of each rule including wildcards"""
  counts = {}  # a dictionary of label -> count.
  for row in rows:
      num_of_rule=row["rule_number"]  
     # the number of wildCard in source IP is 32-number of relavent bits
      s_wildCard=32-int(row["s_wildCard"])
      # same for dest's wildCard
      d_wildCard=32-int(row["d_wildCard"])
       # bit is a wild card so given bit we count only half of the wild card
      total_wildCard=s_wildCard+d_wildCard
      if(bit!='all' and row["IP"][bit]=='*'):
        total_wildCard=total_wildCard-1

      counts[num_of_rule] =2**total_wildCard
  return counts

###Compute Entropy for a set of rules
We use the formula we learnt in class :
lets denote for each rule $R_i$ the number $N_{R_i}=2^{|\phi|}$ 
Where $|\phi|$ is the number of the don't cares.
Hence:
>$H(R)=-\sum_{i=0}^n \frac{N_{R_i}}{N_{total}}\cdot log_2( \frac{N_{R_i}}{N_{total}})$


In [ ]:
def compute_HR(rows,bit,criteria="IG"):
  if(criteria=='IG'):
     counts = update_counts(rows,bit)
     Ntotal=0
     for num in counts:
        Ntotal+=counts[num]
     HR = 0
     for lbl in counts:
         prob_of_lbl = counts[lbl] / float(Ntotal)
         HR -= prob_of_lbl*math.log2(prob_of_lbl)
   
  elif(criteria=='entropy'):
    #  we want to count the entropy without the effect of wildcards with a given bit
     counts = update_counts(rows,-1)
     Ntotal=0
     for num in counts:
        Ntotal+=counts[num]
     HR = 0
     for lbl in counts:
         prob_of_lbl = counts[lbl] / float(Ntotal)
         HR -= prob_of_lbl*math.log2(prob_of_lbl)
  return HR

In [ ]:
# Check the HR of our set
rows=get_data_rows(data)
HR=compute_HR(rows,'all')
HR

1.094003600304795

###Compute Information Gain
We use the formula we learnt in class :
>$IG(R|b_j)=H(R)-H(R|b_j)$

When:
>$H(R|b_j)=p(b_j=1)\cdot H(R|b_j =1)+p(b_j=0)\cdot H(R|b_j =0)$

In [ ]:
def info_gain(left, right, HR,bit,take='all'):
    """Information Gain.
    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    if(take=='left'):
      p=1
    elif(take=='right'):
      p=0
    else:
      p=0.5
    return HR - p * compute_HR(left,bit) - (1 - p) * compute_HR(right,bit)


## Creates a Question class
A Question is used to partition a dataset. This class just records a 'bit number' (e.g. bit number 32) and a bit value (e.g., 0 or 1 ). The 'match' method is used to compare the feature value in an example to the feature value stored in the question. See the demo below.


In [ ]:
class Question:
    def __init__(self,bit, value):
        self.value = value
        self.bit=bit;
    
    def match(self, example):
        # we check if the IP we got in bit self.bit is equals to the value the Quistion got
        val=example["IP"][self.bit]
        return val == str(self.value)

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        return "Is IP in bit %s %s %s?" % (
            self.bit, condition, str(self.value))

In [ ]:
# Test if we build the Questions correctly
# Let's write a question
q=Question(0,1)
print("Question:",q)
# get rows from the data
rows=get_data_rows(data)
# choose the first row to be the example
example=rows[0]
print("example row: ",example)
print("Match result: ",q.match(example))
print("\n")
example=rows[3]
q=Question(0,0)
print("Question:",q)
print("example row: ",example)
print("Match result: ",q.match(example))

Question: Is IP in bit 0 == 1?
example row:  {'IP': '1011010010101011100100111011101101010011101001110110111010011010', 's_wildCard': '32', 'd_wildCard': '32', 'rule_number': 0}
Match result:  True


Question: Is IP in bit 0 == 0?
example row:  {'IP': '1011010010101011100100110011111100011001101100101000011101010110', 's_wildCard': '32', 'd_wildCard': '32', 'rule_number': 3}
Match result:  False


## Partitions the dataset
For each row in the dataset, check if it matches the question. If so, add it to 'true rows', otherwise, add it to 'false rows'.


In [ ]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [ ]:
#checks the partition anf information gain for different questions:
rows=get_data_rows(data)
bit1=4
q1=Question(bit1,1)
HR=compute_HR(rows,'all')
true_rows, false_rows = partition(rows,q1)
print("iformation gain to question",q1,": ", info_gain(true_rows, false_rows, HR,bit1))
bit2=19
q2=Question(bit2,0)
true_rows, false_rows = partition(rows,q2)
print("iformation gain to question",q2,": ", info_gain(true_rows, false_rows, HR,bit2))


iformation gain to question Is IP in bit 4 == 1? :  0.525750026229028
iformation gain to question Is IP in bit 19 == 0? :  0.5469975814466264


### Find Best Question to split The data
we use different critirias of spliting- we iterate over all values of bits , calculate the gain according the cretiria and find the best question to ask in the current stage by the optimum critetia.


possible criterias:
*   IG_best_branch
*   IG_best_level
*   entropy_best_branch
*   entropy_best_level





In [ ]:
def find_best_split(rows,criterita="IG_best_level"):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = compute_HR(rows,-1)
    n_features =64  # number of columns  
    values={0,1}
    for bit in range(n_features):  # for each feature
      for val in values:  # for each value
        question = Question(bit,val)
        # print(question,bit,val)
        true_rows, false_rows = partition(rows,question)
        # print (true_rows)
        if len(true_rows) == 0 or len(false_rows) == 0:
              continue
        
        # 1.3.1 For each branch in the tree you take the best bit, so different branches may use
        # different sequences of bits
        if(criterita=='IG_best_branch'):      
          gain_left = info_gain(true_rows, false_rows, current_uncertainty,bit,'left')
          gain_right = info_gain(true_rows, false_rows, current_uncertainty,bit,'right')
          gain=gain_left if gain_left>gain_right else gain_left
        
        # 1.3.2.Branches at the same level will use the bit that provides the best IG among all the
        # tested bits, so all paths will use the same order of bits
        elif(criterita=='IG_best_level'):      
          gain = info_gain(true_rows, false_rows, current_uncertainty,bit)
        
        # criteria is changed so the bit with the highest entropy is selected 
        # the best bit for each branch 
        elif(criterita=='entropy_best_branch'):
          best_bit_true=compute_HR(true_rows,bit)
          best_bit_false=compute_HR(false_rows,bit)
          gain=best_bit_true if best_bit_true>best_bit_false else best_bit_false
        
        # bit for ll branches
        elif(criterita=='entropy_best_level'):
          gain=compute_HR(rows,bit)
      

        if gain >= best_gain:
              best_gain, best_question = gain, question
    
    return best_gain, best_question

In [ ]:
#Demo- lets find the best question to ask first , with creteria IG_best_level
rows=get_data_rows(data)
best_gain, best_question = find_best_split(rows,'IG_best_level')
print(best_question)

Is IP in bit 19 == 0?


## Building Decision tree
In this section, I created all the classess and method for creating the decision tree, and display it

### Classes of decision tree

In [ ]:
class Leaf:
    def __init__(self, rows):
        self.predictions = update_counts(rows,-1)
    
    def height(self):
      return 1

In [ ]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
      
    def height(self):
      left_depth = self.true_branch.height()
      right_depth = self.false_branch.height() 
      return max(left_depth, right_depth) + 1
    

### Methods for decision tree

In [ ]:
def build_tree(rows,stopping_point,criteria):
    """Builds the tree.
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows,criteria)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)
 
    true_rows, false_rows = partition(rows, question)
    true_branch= Leaf(true_rows) if (len(true_rows)<=stopping_point) else build_tree(true_rows,stopping_point,criteria)
    false_branch=Leaf(false_rows) if (len(false_rows)<=stopping_point) else build_tree(false_rows,stopping_point,criteria)
    return Decision_Node(question, true_branch, false_branch)

In [ ]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict: {Rule number,number of accur} ", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [ ]:
rows=get_data_rows(data)
criterias=['IG_best_branch','IG_best_level','entropy_best_branch','entropy_best_level']
for criteria in criterias:
  # stopping point -1 means no stopping point
  my_tree= build_tree(rows,-1,criteria)
  print(["criteria",criteria,"my_tree depth:",my_tree.height()])
  my_tree_16= build_tree(rows,16,criteria)
  print(["criteria",criteria,"my_tree_16 depth:",my_tree_16.height()])
  my_tree_32= build_tree(rows,32,criteria)
  print(["criteria",criteria,"my_tree_32 depth:",my_tree_32.height()])
  my_tree_64= build_tree(rows,64,criteria)
  print(["criteria",criteria,"my_tree_64 depth:",my_tree_64.height()])
  my_tree_96= build_tree(rows,96,criteria)
  print(["criteria",criteria,"my_tree_96 depth:",my_tree_96.height()])
  my_tree_128= build_tree(rows,128,criteria)
  print(["criteria",criteria,"my_tree_128 depth:",my_tree_128.height()])
  print("---------------------------------------------------------")
  # print_tree(my_tree)
  # print("---------------------------------------------------------")


['criteria', 'IG_best_branch', 'my_tree depth:', 27]
['criteria', 'IG_best_branch', 'my_tree_16 depth:', 26]
['criteria', 'IG_best_branch', 'my_tree_32 depth:', 25]
['criteria', 'IG_best_branch', 'my_tree_64 depth:', 19]
['criteria', 'IG_best_branch', 'my_tree_96 depth:', 19]
['criteria', 'IG_best_branch', 'my_tree_128 depth:', 15]
---------------------------------------------------------
['criteria', 'IG_best_level', 'my_tree depth:', 19]
['criteria', 'IG_best_level', 'my_tree_16 depth:', 11]
['criteria', 'IG_best_level', 'my_tree_32 depth:', 7]
['criteria', 'IG_best_level', 'my_tree_64 depth:', 5]
['criteria', 'IG_best_level', 'my_tree_96 depth:', 5]
['criteria', 'IG_best_level', 'my_tree_128 depth:', 5]
---------------------------------------------------------
['criteria', 'entropy_best_branch', 'my_tree depth:', 27]
['criteria', 'entropy_best_branch', 'my_tree_16 depth:', 26]
['criteria', 'entropy_best_branch', 'my_tree_32 depth:', 24]
['criteria', 'entropy_best_branch', 'my_tree_6

In [ ]:
# Demo - example for decision tree
rows=get_data_rows(data)
criterias=['IG_best_branch','IG_best_level','entropy_best_branch','entropy_best_level']
criteria='IG_best_level'
my_tree_16= build_tree(rows,16,criteria)
print(["criteria",criteria,"my_tree_16:"])
print_tree(my_tree_16)
print("---------------------------------------------------------")
 


In [ ]:
def classify(row, node):
    """See the 'rules of recursion' above."""
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [ ]:
def print_leaf(counts):
    
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        if(int(counts[lbl] / total * 100)!=0):
          probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [ ]:
rows=get_data_rows(data)
great_indexes=[0,92,108,109,200,600]
for index in great_indexes:
  print("try to predict rule number:",index)
  print(print_leaf(classify(rows[index], my_tree)))

print("-----------------------------------------------")

medium_indexes=[1,14,15,17,40,45]
for index in medium_indexes:
  print("try to predict rule number:",index)
  print(print_leaf(classify(rows[index], my_tree)))

print("-----------------------------------------------")
low_indexes=[10,20,30,100,500]
for index in low_indexes:
  print("try to predict rule number:",index)
  print(print_leaf(classify(rows[index], my_tree)))

#######

try to predict rule number: 0
{0: '100%'}
try to predict rule number: 92
{92: '100%'}
try to predict rule number: 108
{108: '100%'}
try to predict rule number: 109
{109: '100%'}
try to predict rule number: 200
{200: '100%'}
try to predict rule number: 600
{600: '100%'}
-----------------------------------------------
try to predict rule number: 1
{1: '50%', 513: '50%'}
try to predict rule number: 14
{14: '50%', 15: '50%'}
try to predict rule number: 15
{14: '50%', 15: '50%'}
try to predict rule number: 17
{17: '50%', 130: '50%'}
try to predict rule number: 40
{40: '25%', 41: '25%', 300: '25%', 484: '25%'}
try to predict rule number: 45
{45: '33%', 46: '33%', 516: '33%'}
-----------------------------------------------
try to predict rule number: 10
{8: '9%', 10: '9%', 85: '9%', 94: '9%', 104: '9%', 249: '9%', 267: '9%', 287: '9%', 297: '9%', 537: '9%', 539: '9%'}
try to predict rule number: 20
{20: '7%', 22: '7%', 23: '7%', 142: '7%', 143: '7%', 144: '7%', 146: '7%', 147: '7%', 148: '7%'